**Клонирование репозитония**

In [2]:
!git clone https://github.com/pyramidheadshark/BIV_2024.git

Cloning into 'BIV_2024'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 130 (delta 49), reused 110 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (130/130), 22.27 MiB | 14.22 MiB/s, done.
Resolving deltas: 100% (49/49), done.


### Тренировка модели

In [6]:
!pip install datasets > None
!pip install onnxruntime > None
!pip install optimum > None
!pip install onnx > None

In [26]:
!python3 src/models_dev/train.py \
                   --file_path data/processed/payments_categorized_v3.tsv\
                   --model_name sergeyzh/rubert-tiny-turbo \
                   --output_dir ./results \
                   --batch_size 512 \
                   --epochs 50 \
                   --learning_rate 5e-5 \
                   --save_path models/trained_model

2024-11-16 18:10:20.890282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 18:10:20.911122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 18:10:20.917143: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 18:10:20.931111: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-16 18:10:22.386377: W tensorflow/comp

### Конвертация в onnx

In [27]:
!python3 src/models_dev/convert_to_onnx.py \
                            --model_path models/trained_model \
                            --onnx_path models/onnx_model

2024-11-16 18:19:25.224160: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 18:19:25.244085: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 18:19:25.250011: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 18:19:25.264085: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-16 18:19:26.418803: W tensorflow/comp

### Квантирование

In [28]:
!python3 src/models_dev/quantize.py --onnx_model_path models/onnx_model/model.onnx \
                                --save_dir models/onnx_model \
                                --max_length 128

Модель квантована и сохранена: models/onnx_model/model_quantized.onnx


### Тестирование скорости инференса

In [29]:
!python3 src/models_dev/q_infer.py --file_path data/raw/payments_main.tsv \
                 --model_path models/onnx_model/model_quantized.onnx \
                 --tokenizer_path models/onnx_model \
                 --output_file data/predictions.tsv \
                 --batch_size 64 \
                 --max_length 128

Inferencing: 100% 391/391 [02:02<00:00,  3.19it/s]
Predictions saved to data/predictions.tsv


In [30]:
!zip -r models.zip models/onnx_model

updating: models/onnx_model/ (stored 0%)
updating: models/onnx_model/tokenizer_config.json (deflated 73%)
updating: models/onnx_model/vocab.txt (deflated 64%)
updating: models/onnx_model/model_quantized.onnx (deflated 39%)
updating: models/onnx_model/model.onnx (deflated 8%)
updating: models/onnx_model/special_tokens_map.json (deflated 80%)
updating: models/onnx_model/config.json (deflated 56%)
updating: models/onnx_model/tokenizer.json (deflated 73%)


### Тестирование точности квантированной модели

In [31]:
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv("data/processed/payments_categorized_v3.tsv", sep='\t', header=None, names=['date', 'amount', 'description', 'category'])
data.drop(index=0, inplace=True)
data = data.dropna(subset=['category'])
train_data, val_data = train_test_split(data, test_size=0.2, stratify=data['category'], random_state=42)

# Сохранение val_data для тестирования
val_data.to_csv("data/processed/val_data.tsv", sep='\t', index=False)

In [32]:
!python3 src/models_dev/q_infer_val.py \
    --val_data_path data/processed/val_data.tsv \
    --model_path models/onnx_model/model_quantized.onnx \
    --tokenizer_path models/onnx_model \
    --batch_size 64 \
    --max_length 128

Inferencing: 100% 56/56 [00:17<00:00,  3.27it/s]
True labels: [2, 3, 4, 4, 6, 8, 7, 4, 4, 6]
Predictions: [2, 3, 4, 4, 6, 1, 7, 4, 4, 6]
Validation Accuracy: 0.8458
Validation F1 Score: 0.8392
